### 해당모델 관련한 내용

#### 랜덤 포레스트  
    
- 결정 트리에는 과적합에 취약하다는 단점이 있는데, 트리의 깊이가 깊어짐에 따라 학습 데이터에만 적합한 과도하게 불규칙한 패턴을 학습하는 경향이 커지기 때문이다.  
- 결정 트리의 분산을 줄이고자 랜덤 포레스트에서는 학습 데이터를 다르게 샘플링한 후 각각의 서브데이터에서 깊이가 깊은 결정 트리를 학습하고 이 결과를 최종적으로 종합하여 출력값을 계산한다.  

      
- 즉, 부트스트래핑 기반 샘플링을 활용한 의사결정나무 생성 이후 배깅 기반 나무들을 모아 앙상블 학습하여 숲을 형성하게 되면 이를 랜덤포레스트라고 일컫는다.  
    - 부트스트래핑: 단순 복원 임의추출법(랜덤샘플링)으로 크기가 동일한 여러 개의 표본자료를 생성한다.  
    - 배깅: 여러 부트스트랩 자료를 생성하여 학습하는 모델링으로 분류기를 생성한 후 그 결과를 앙상블 하는 방법이다.  
               + 추출한 각 샘플별 모델링 학습 뒤 결과들을 집계, 최종 결과를 만들어 내는 방식으로 범주형 데이터인 경우 다수결 투표방식,  
            연속형 데이터인 경우 평균으로 결과를 집계  
    - 앙상블 학습: 여러 모델을 학습시켜 결합하는 방식의 학습방법으로 일반화 성능을 향상시켜 과적합을 해결할 수 있음.  

### 랜덤포레스트 모델 파라미터

- sklearn.ensemble.RandomForestClassifier 클래스는 랜덤 포레스트 분류 모델을 구현한다.

<하이퍼파라미터>
- n_estimators (default: 100) -> 트리 개수
- bootstrap (default: True) -> 부트스트랩 샘플 사용 여부  
  - True 개별 트리를 학습할 때 부트스트랩 샘플을 사용
  - False 개별 트리를 학습할 때 부트스트랩 샘플 대신 항상 전체 데이터셋 사용. 이때도 max_features에 따라 트리별  
    랜덤성이 발생하므로 랜덤 포레스트 모델에서의 각각의 트리는 서로 다른 구조를 가짐.  
- ood_score (default: False) -> bootstrap=True 일 때만 유효하며, 일반화 점수를 계산할 때 OOB샘플을 사용할지 선택
- max_samples (default: None) -> bootstrap=True 일 때만 유효하며, 각각의 베이스 학습기를 학습하기 위해 추출할 샘플 개수를 지정
  - None: 전체 샘플 추출
  - int: max_samples 건 추출
  - float: n과 max_samples의 곱에 해당하는 샘플 개수를 추출
- (회귀) criterion (defalut: 'squared_error') 분기 기준 'squared_error', 'absolute_error', 'poisson'은 DecisionTreeRegressor 클래스에서와 의미가 같다.

### 해당모델 관련 예시코드 (당뇨진척정도-회귀)

In [1]:
## 1. 라이브러리 및 데이터 확인
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes

#diabetes 데이터셋 로드
diabetes = load_diabetes()
x = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
y = pd.DataFrame(diabetes.target)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                   test_size = 0.2,
                                                   random_state = 2023)

x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_test.rename(columns={'index':'cust_id'}, inplace=True)
x_train.rename(columns={'index':'cust_id'}, inplace=True)
y_train.columns = ['cust_id', 'target']

#### 당뇨병 환자의 질병 진행정도를 예측

In [2]:
#데이터 설명
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [3]:
## 데이터 탐색(EDA)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(353, 11)
(89, 11)
(353, 2)


In [4]:
# 초기 데이터 확인
print(x_train.head())
print(x_test.head())
print(y_train.head())

   cust_id       age       sex       bmi        bp        s1        s2  \
0        4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596   
1      318  0.088931 -0.044642  0.006728  0.025315  0.030078  0.008707   
2      301 -0.001882  0.050680 -0.024529  0.052858  0.027326  0.030001   
3      189 -0.001882 -0.044642 -0.066563  0.001215 -0.002945  0.003070   
4      288  0.070769  0.050680 -0.016984  0.021872  0.043837  0.056305   

         s3        s4        s5        s6  
0  0.008142 -0.002592 -0.031988 -0.046641  
1  0.063367 -0.039493  0.009434  0.032059  
2  0.030232 -0.002592 -0.021395  0.036201  
3  0.011824 -0.002592 -0.020292 -0.025930  
4  0.037595 -0.002592 -0.070209 -0.017646  
   cust_id       age       sex       bmi        bp        s1        s2  \
0      280  0.009016  0.050680  0.018584  0.039087  0.017694  0.010586   
1      412  0.074401 -0.044642  0.085408  0.063187  0.014942  0.013091   
2       68  0.038076  0.050680 -0.029918 -0.040099 -0.033216 -0.02417

In [5]:
#변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  353 non-null    int64  
 1   age      353 non-null    float64
 2   sex      353 non-null    float64
 3   bmi      353 non-null    float64
 4   bp       353 non-null    float64
 5   s1       353 non-null    float64
 6   s2       353 non-null    float64
 7   s3       353 non-null    float64
 8   s4       353 non-null    float64
 9   s5       353 non-null    float64
 10  s6       353 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 30.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  89 non-null     int64  
 1   age      89 non-null     float64
 2   sex      89 non-null     float64
 3   bmi      89 non-null     float64
 4   bp      

In [6]:
# x_train 과 x_test 데이터의 기초통계량 비교

print(x_train.describe()) 
print(x_test.describe())
print(y_train.describe())

          cust_id         age         sex         bmi          bp          s1  \
count  353.000000  353.000000  353.000000  353.000000  353.000000  353.000000   
mean   212.634561    0.000804    0.000724    0.000640   -0.000326    0.001179   
std    126.668903    0.047617    0.047673    0.048141    0.046585    0.047891   
min      0.000000   -0.107226   -0.044642   -0.084886   -0.112399   -0.126781   
25%    105.000000   -0.038207   -0.044642   -0.035307   -0.033213   -0.033216   
50%    210.000000    0.005383   -0.044642   -0.006206   -0.005670   -0.002945   
75%    322.000000    0.038076    0.050680    0.030440    0.032201    0.027326   
max    441.000000    0.110727    0.050680    0.170555    0.125158    0.153914   

               s2          s3          s4          s5          s6  
count  353.000000  353.000000  353.000000  353.000000  353.000000  
mean     0.001110   -0.000452    0.000901    0.001446    0.000589  
std      0.048248    0.048600    0.048045    0.047160    0.048122 

In [7]:
# y데이터도 구체적으로 살펴보기
print(y_train.head())

   cust_id  target
0        4   135.0
1      318   109.0
2      301    65.0
3      189    79.0
4      288    80.0


In [8]:
# y데이터도 구체적으로 살펴보기
print(y_train.describe().T)

         count        mean         std   min    25%    50%    75%    max
cust_id  353.0  212.634561  126.668903   0.0  105.0  210.0  322.0  441.0
target   353.0  152.943343   75.324692  37.0   90.0  141.0  208.0  346.0


In [9]:
## 데이터 전처리 및 분리
## 1) 결측치 2) 이상치 3) 변수처리

#결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())


#결측치 제거
#df = df.dropna()
#print(df)

#참고사항
#print(df.dropna().shape) #행 기준으로 삭제

#결측치 대체(평균값, 중앙값, 최빈값)
#연속형 변수: 중앙값, 평균값
# -df['변수명'].median()
# -df['변수명'].mean()
# 범주형 변수 : 최빈값

#df['변수명'] = df['변수명'].fillna(대체할 값)

cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
target     0
dtype: int64


In [10]:
#이상치 대체
#(참고) df['변수명'] = np.where(df['변수명']>=5, 대체할 값, df['변수명'])

#변수 처리

#불필요한 변수 제거
#df = df.drop(columns = ['변수1', '변수2'])
#df = df.drop(['변수1', '변수2'], axis = 1)

#필요시 변수 추가 (파생변수 생성)
#df['파생변수명'] = df['A'] * df['B'] 파생변수 생성 수식

#원핫인코딩(가변수 처리)
#x_train = pd.get_dummies(x_train)
#x_test = pd.get_dummies(x_test)
#print(x_train.info())
#print(x_test.info())

In [11]:
#변수 처리

# 불필요한 변수(columns) 제거
# cust_id 는 불필요한 변수이므로 제거한다.
# 단, test 셋의 cust_id가 나중에 제출이 필요하기 때문에 별도로 저장

cust_id = x_test['cust_id'].copy()

#각 데이터에서 cust_id 변수 제거
x_train = x_train.drop(columns = ['cust_id']) #df = df.drop(columns = ['변수1', '변수2'])
x_test = x_test.drop(columns = ['cust_id'])


In [12]:
## 데이터 분리
# 데이터를 훈련 세트와 검증용 세트로 분할 (80%훈련, 20%검증용)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['target'],
                                                test_size = 0.2,
                                                random_state=2023) 
#####stratify는 분류모델에서만 넣는 것, 연속형모델은 오류남
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(282, 10)
(71, 10)
(282,)
(71,)


In [21]:
##모델링 및 성능평가
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 200,
                              random_state = 2023)
model.fit(x_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=2023)

In [22]:
#모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [23]:
#모델 성능 평가 (평균 제곱 오차 및 R-squared)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_val, y_pred) #(실제값, 예측값)
r2 = r2_score(y_val, y_pred)            #(실제값, 예측값)

In [24]:
#MSE
print(mse)

3703.9832676056344


In [25]:
#r2 score
print(r2)

0.2714935295517188


In [26]:
#RMSE
rmse = mse**0.5
print(rmse)

60.86035875350748
